In [200]:
import psycopg2
import pandas as pd 
# from sqlalchemy.types import Integer, Text, String, DateTime
import sqlalchemy as s
import matplotlib
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json

with open("config.json") as config_file:
    config = json.load(config_file)

database_connection_string = 'postgres+psycopg2://{}:{}@{}:{}/{}'.format(config['user'], config['password'], config['host'], config['port'], config['database'])

dbschema='augur_data'
engine = s.create_engine(
    database_connection_string,
    connect_args={'options': '-csearch_path={}'.format(dbschema)})

In [201]:
import datetime 

current = datetime.date.today()
today = "'" + str(current) + "'"
#print(current)

first_current = current.replace(day=1)
last_month = first_current - datetime.timedelta(days=1)
end_date = "'" + str(last_month) + "'"

start = last_month - datetime.timedelta(days=180)
start_date = "'" + str(start) + "'"

# Note - Use this get current status, comment out to look at whether past month has finished collection.
#end_date = today
print(start_date, end_date)

'2020-08-04' '2021-01-31'


In [202]:
missing_query = f"""
SELECT
    * 
FROM
    (
    SELECT
        repo_info.repo_id,
        repo.repo_name,
repo.repo_git, 
        MAX ( pull_request_count ) AS max_pr_count,
        COUNT ( * ) AS meta_count 
    FROM
        repo_info,
        repo -- WHERE issues_enabled = 'true' 
    WHERE
        pull_request_count >= 1
        AND repo.repo_id = repo_info.repo_id 
    GROUP BY
        repo_info.repo_id,
        repo.repo_name, 
 repo.repo_git 
    ORDER BY
        repo_info.repo_id,
        repo.repo_name 
    ) yy
    LEFT OUTER JOIN (
    SELECT A
        .repo_id,
        A.repo_name,
        b.pull_request_count,
        d.repo_id AS pull_request_repo_id,
        e.last_collected,
        f.last_pr_collected,
        COUNT ( * ) AS pull_requests_collected,
        ( b.pull_request_count - COUNT ( * ) ) AS pull_requests_missing,
        ABS ( CAST ( ( COUNT ( * ) ) + 1 AS DOUBLE PRECISION ) / CAST ( b.pull_request_count + 1 AS DOUBLE PRECISION ) ) AS ratio_abs,
        ( CAST ( ( COUNT ( * ) ) + 1 AS DOUBLE PRECISION ) / CAST ( b.pull_request_count + 1 AS DOUBLE PRECISION ) ) AS ratio_prs 
    FROM
        augur_data.repo A,
        augur_data.pull_requests d,
        augur_data.repo_info b,
        ( SELECT repo_id, MAX ( data_collection_date ) AS last_collected FROM augur_data.repo_info GROUP BY repo_id ORDER BY repo_id ) e,
        ( SELECT repo_id, MAX ( data_collection_date ) AS last_pr_collected FROM augur_data.pull_requests GROUP BY repo_id ORDER BY repo_id ) f 
    WHERE
        A.repo_id = b.repo_id 
        AND LOWER ( A.repo_git ) LIKE'%github.com%' 
        AND A.repo_id = d.repo_id 
        AND b.repo_id = d.repo_id 
        AND e.repo_id = A.repo_id 
        AND b.data_collection_date = e.last_collected 
        AND f.repo_id = A.repo_id -- AND d.pull_request_id IS NULL
        
    GROUP BY
        A.repo_id,
        d.repo_id,
        b.pull_request_count,
        e.last_collected,
        f.last_pr_collected 
    ORDER BY
        ratio_abs desc
    ) zz ON yy.repo_id = zz.repo_id 
ORDER BY
    ratio_abs;
    """
missing_data = pd.read_sql_query(missing_query, con=engine)
print(missing_data)

      repo_id                    repo_name  \
0       28141                boarding-pass   
1       27548                      pairist   
2       25888   greenplum-database-release   
3       30832                   BlockHound   
4       30932                         salt   
...       ...                          ...   
2239    26440                     docs-ksm   
2240    25579                      netdata   
2241    25565  bosh-linux-stemcell-builder   
2242    26291      datadog-firehose-nozzle   
2243    26811            pcfbackup-release   

                                               repo_git  max_pr_count  \
0            https://github.com/concourse/boarding-pass            12   
1                 https://github.com/pivotal-cf/pairist            16   
2     https://github.com/greenplum-db/greenplum-data...           115   
3                 https://github.com/reactor/BlockHound           121   
4                     https://github.com/saltstack/salt         36211   
...      

In [203]:
df = missing_data.loc[missing_data['ratio_abs'] < 1]
#print(df)
df1 = df[['repo_id', 'last_pr_collected', 'pull_requests_missing', 'ratio_abs', 'pull_requests_collected', 'repo_git']]
#df1 = df[['pull_requests_missing', 'repo_git']]
print(df1)
#missing_data.to_csv('/Users/dafoster/gitrepos/augur-pivotal/code/health_scripts/output/missing.csv')

   repo_id  repo_id   last_pr_collected  pull_requests_missing  ratio_abs  \
0    28141  28141.0 2020-12-07 18:57:41                    1.0   0.923077   
1    27548  27548.0 2021-02-04 03:13:24                    1.0   0.941176   
2    25888  25888.0 2021-02-04 08:53:25                    1.0   0.991379   
3    30832  30832.0 2020-12-27 15:49:03                    1.0   0.991803   
4    30932  30932.0 2021-01-23 17:27:43                   41.0   0.998868   

   pull_requests_collected                                           repo_git  
0                     11.0         https://github.com/concourse/boarding-pass  
1                     15.0              https://github.com/pivotal-cf/pairist  
2                    114.0  https://github.com/greenplum-db/greenplum-data...  
3                    120.0              https://github.com/reactor/BlockHound  
4                  36170.0                  https://github.com/saltstack/salt  


In [204]:
df = missing_data.loc[missing_data['ratio_abs'] < 1]
#print(df)
df1 = df[['repo_id', 'last_pr_collected', 'pull_requests_missing', 'ratio_abs', 'repo_git']]
#df1 = df[['pull_requests_missing', 'repo_git']]
print(df1)

for index, repo in df1.iterrows():
    if 'saltstack' in repo['repo_git']:
        print(repo['ratio_abs'], repo['repo_git'])

   repo_id  repo_id   last_pr_collected  pull_requests_missing  ratio_abs  \
0    28141  28141.0 2020-12-07 18:57:41                    1.0   0.923077   
1    27548  27548.0 2021-02-04 03:13:24                    1.0   0.941176   
2    25888  25888.0 2021-02-04 08:53:25                    1.0   0.991379   
3    30832  30832.0 2020-12-27 15:49:03                    1.0   0.991803   
4    30932  30932.0 2021-01-23 17:27:43                   41.0   0.998868   

                                            repo_git  
0         https://github.com/concourse/boarding-pass  
1              https://github.com/pivotal-cf/pairist  
2  https://github.com/greenplum-db/greenplum-data...  
3              https://github.com/reactor/BlockHound  
4                  https://github.com/saltstack/salt  
0.9988677786369159 https://github.com/saltstack/salt


In [205]:
for index, repo in df1.iterrows():
    print(repo['last_pr_collected'], repo['repo_git'])

2020-12-07 18:57:41 https://github.com/concourse/boarding-pass
2021-02-04 03:13:24 https://github.com/pivotal-cf/pairist
2021-02-04 08:53:25 https://github.com/greenplum-db/greenplum-database-release
2020-12-27 15:49:03 https://github.com/reactor/BlockHound
2021-01-23 17:27:43 https://github.com/saltstack/salt


In [206]:
#Repos for gathering data

def get_commits_by_repo(start_date, end_date, engine):
    import pandas as pd

    repo_list_commits = pd.DataFrame()
    repo_list_commits_query = f"""
            SELECT COUNT(DISTINCT commits.cmt_commit_hash), repo.repo_id, repo.repo_name, repo.repo_path from repo, commits
            WHERE 
                repo.repo_id = commits.repo_id
                AND commits.cmt_author_timestamp >= {start_date}
                AND commits.cmt_author_timestamp <= {end_date}
                AND cmt_author_name NOT LIKE '%%utomation%%'
                AND cmt_author_name NOT LIKE '%%ipeline%%'
                AND cmt_author_name NOT LIKE '%%Cloud Foundry%%'
                AND cmt_author_name NOT LIKE 'snyk%%'
                AND cmt_author_name NOT LIKE '%%bot'
                AND cmt_author_name NOT LIKE 'dependabot%%'
                AND cmt_author_name NOT LIKE '%%Bot'
                AND cmt_author_name NOT LIKE '%%BOT'
                AND cmt_author_name != 'cfcr'
                AND cmt_author_name != 'CFCR'
                AND cmt_author_name != 'Travis CI'
                AND cmt_author_name != 'Cloud Foundry London'
                AND cmt_author_name != 'pivotal-rabbitmq-ci'
                AND cmt_author_name != 'Bitnami Containers'
                AND cmt_author_name != 'Spring Operator'
                AND cmt_author_name != 'Spring Buildmaster'
            GROUP BY repo.repo_id
            ORDER BY COUNT(DISTINCT commits.cmt_commit_hash);
            """
    repo_list_commits = pd.read_sql_query(repo_list_commits_query, con=engine)
    
    return repo_list_commits
    
repo_list_commits = get_commits_by_repo(start_date, end_date, engine)

top = repo_list_commits.loc[repo_list_commits['count'] > 60]



In [207]:
print(top[0:157])

      count  repo_id                   repo_name                    repo_path
1109     62    30662      spring-cloud-zookeeper     github.com/spring-cloud/
1110     63    30650            spring-cloud-gcp     github.com/spring-cloud/
1111     63    28028            captive-web-view           github.com/vmware/
1112     63    25635           spring-javaformat        github.com/spring-io/
1113     66    27526          docs-operating-pas       github.com/pivotal-cf/
...     ...      ...                         ...                          ...
1243   1051    25432             rabbitmq-server         github.com/rabbitmq/
1244   1096    26994  pas-nsx-t-deployment-locks          github.com/pivotal/
1245   1487    30932                        salt        github.com/saltstack/
1246   2636    25760                 spring-boot  github.com/Spring-projects/
1247   5506    27645                     cvelist       github.com/pivotal-cf/

[139 rows x 4 columns]


In [208]:
print('repo_id ratio last_pr_collected              repo_url')
i = 0
for index, repo in top.iterrows():
    repo_id = repo['repo_id']

    try:
        #ratio_abs = round(missing_data.loc[missing_data['pull_request_repo_id'] == repo_id]['ratio_abs'].values[0], 2)
        ratio_abs = format(missing_data.loc[missing_data['pull_request_repo_id'] == repo_id]['ratio_abs'].values[0], '.2f')
    except:
        ratio_abs = 'NA  '
    
    try:
        last_pr = missing_data.loc[missing_data['pull_request_repo_id'] == repo_id]['last_pr_collected'].values[0]
    except:
        last_pr = 'UNKNOWN'
        
    repo_info = str(repo_id) + '   ' + str(ratio_abs) + '  ' + str(last_pr) + '  ' + repo['repo_path'] + repo['repo_name'] 
    print(repo_info)
    i+=1
    
print('Number of repos:', i)

repo_id ratio last_pr_collected              repo_url
30662   1.00  2020-12-07T22:34:47.000000000  github.com/spring-cloud/spring-cloud-zookeeper
30650   1.00  2021-02-04T10:19:32.000000000  github.com/spring-cloud/spring-cloud-gcp
28028   1.00  2020-12-27T11:15:07.000000000  github.com/vmware/captive-web-view
25635   1.00  2020-12-07T07:56:10.000000000  github.com/spring-io/spring-javaformat
27526   1.00  2021-01-28T01:49:04.000000000  github.com/pivotal-cf/docs-operating-pas
25531   1.00  2020-12-08T00:09:51.000000000  github.com/rabbitmq/rabbitmq-common
30354   1.00  2021-02-05T08:51:14.000000000  github.com/saltstack/salt-bootstrap
30661   1.00  2021-01-28T04:56:14.000000000  github.com/spring-cloud/spring-cloud-stream
27939   1.01  2021-01-28T02:04:56.000000000  github.com/vmware/nsx-powerops
26978   1.00  2021-01-28T05:29:15.000000000  github.com/pivotal/LicenseFinder
30630   1.00  2021-02-05T08:44:25.000000000  github.com/spring-cloud/spring-cloud-stream-binder-kafka
25797   1.0

25827   1.00  2020-12-07T08:52:35.000000000  github.com/Spring-projects/sts4
25726   1.00  2021-02-05T09:38:12.000000000  github.com/Spring-projects/spring-security
27512   1.00  2021-02-05T09:39:16.000000000  github.com/pivotal-cf/docs-ops-manager
30478   1.00  2021-02-04T05:28:56.000000000  github.com/k14s/k14s.io
30833   1.00  2021-02-05T08:59:35.000000000  github.com/reactor/reactor-core
28026   1.00  2020-12-27T12:00:32.000000000  github.com/vmware/network-config-manager
25834   1.00  2020-12-07T08:47:40.000000000  github.com/Spring-projects/spring-boot-data-geode
26004   1.00  2021-02-04T04:50:46.000000000  github.com/SteeltoeOSS/Documentation
27950   1.01  2021-02-05T08:58:43.000000000  github.com/vmware/differential-datalog
27621   1.00  2020-12-07T16:51:03.000000000  github.com/pivotal-cf/docs-ksm
28133   1.00  2021-01-22T09:47:32.000000000  github.com/concourse/ci
30830   1.00  2021-02-04T07:43:49.000000000  github.com/reactor/reactor-netty
27598   1.02  2021-02-05T07:07:23.0

In [209]:
print(start_date, end_date)
print('repo_id ratio last_pr_collected              repo_url')
i = 0
for index, repo in top.iterrows():
    repo_id = repo['repo_id']


        #ratio_abs = round(missing_data.loc[missing_data['pull_request_repo_id'] == repo_id]['ratio_abs'].values[0], 2)
    try:
        ratio_abs = float(format(missing_data.loc[missing_data['pull_request_repo_id'] == repo_id]['ratio_abs'].values[0], '.2f'))
    except:
        ratio_abs = 99.0
        
    try:
        last_pr = missing_data.loc[missing_data['pull_request_repo_id'] == repo_id]['last_pr_collected'].values[0]
    except:
        last_pr = 'UNKNOWN'
    
    if ratio_abs < 1:
        repo_info = str(repo_id) + '   ' + str(ratio_abs) + '  ' + str(last_pr)[0:10] + '  ' + repo['repo_path'] + repo['repo_name'] 
        print(repo_info)
        i+=1
    
print('Number of repos:', i)

'2020-08-04' '2021-01-31'
repo_id ratio last_pr_collected              repo_url
Number of repos: 0


SyntaxError: invalid syntax (<ipython-input-47-e3c0d519daae>, line 1)

In [129]:
print(missing_data)

      repo_id                     repo_name  \
0       30850           tanzu-vs-extensions   
1       30477               kapp-controller   
2       30485             setup-k14s-action   
3       28141                 boarding-pass   
4       26980             workstation-setup   
...       ...                           ...   
2239    25579                       netdata   
2240    27607       docs-cloudbees-core-pks   
2241    26291       datadog-firehose-nozzle   
2242    26408                 kibosh-sample   
2243    25606  prometheus_rabbitmq_exporter   

                                               repo_git  max_pr_count  \
0     https://github.com/steeltoeoss-incubator/tanzu...             6   
1               https://github.com/k14s/kapp-controller            34   
2             https://github.com/k14s/setup-k14s-action            84   
3            https://github.com/concourse/boarding-pass            12   
4          https://github.com/pivotal/workstation-setup           167 

In [149]:
#print(missing_data.loc[missing_data['repo_id'] == 30039])
print(missing_data[(missing_data == 27511).any(axis=1)]['pull_requests_collected'])

1192    58.0
Name: pull_requests_collected, dtype: float64


In [49]:
!cat config.json

{
    "connection_string": "sqlite:///:memory:",
    "database": "panthers_project",
    "host": "vista.osshealth.io",
    "password": "UKPivotal",
    "port": 5433,
    "schema": "augur_data",
    "user": "dawn",
    "user_type": "read_only"
}


In [ ]:
check_prs = pd.DataFrame()
check_prs_query = f"""
select 'pull requets collected' as data_point, count(*) as count from pull_requests
union 
SELECT
data_point, 
SUM ( COUNT ) AS COUNT 
FROM
(
SELECT 'platform metadata' as data_point, 
repo_id,
SUM ( last_collected ) AS COUNT 
FROM
( SELECT repo_id, MAX ( data_collection_date ), COUNT ( * ) AS last_collected FROM augur_data.repo_info GROUP BY repo_id ORDER BY repo_id ) A 
GROUP BY
repo_id
) b group by data_point;
"""
check_prs = pd.read_sql_query(check_prs_query, con=engine)
print(check_prs)